# MASA Hackathon 2022
The endemic signified several things, but the most thrilling aspect was the ability to travel again. Border restrictions are being eased, and there is a trend towards ecotourism. Touchless service delivery, investing, diversifying, and changing to more sustainable tourist models are present, with social distancing and health and hygiene norms presumably in place.

Even at the endemic stage, every country has separate quarantine and healthcare policies. The last thing you will expect on vacation is being delayed at immigration owing to vaccination permits, contracting Covid-19 at a foreign country or having your flight cancelled at the last minute. But what if it that happens? A travel insurance policy will provide you and your family with peace of mind in knowing that you are insured.

## Data Dictionary
Row: 63326 rows

Columns: 11 columns
- Agency: 16 types 'XXX'
- Agency Type: Airlines/Travel Agency
- Distribution Channel: Online/Offline
- Product Name: 26 types
- Claim: Yes/No
- Duration: -2 to 4881
- Destination: 149 countries
- Net Sale: -389 to 810
- Commision: 0 to 283.5
- Gender: F/M
- Age: 0 to 118

## Load Library

In [1]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

In [2]:
# Format code to pep8 standard
# Type in terminal: autopep8 --in-place -a -a Testing01.ipynb

## Data Analysis

In [3]:
# Select colour pastel
colors = sns.color_palette('pastel')
colors

[(0.6313725490196078, 0.788235294117647, 0.9568627450980393),
 (1.0, 0.7058823529411765, 0.5098039215686274),
 (0.5529411764705883, 0.8980392156862745, 0.6313725490196078),
 (1.0, 0.6235294117647059, 0.6078431372549019),
 (0.8156862745098039, 0.7333333333333333, 1.0),
 (0.8705882352941177, 0.7333333333333333, 0.6078431372549019),
 (0.9803921568627451, 0.6901960784313725, 0.8941176470588236),
 (0.8117647058823529, 0.8117647058823529, 0.8117647058823529),
 (1.0, 0.996078431372549, 0.6392156862745098),
 (0.7254901960784313, 0.9490196078431372, 0.9411764705882353)]

In [4]:
# Read data
df = pd.read_csv('Dataset/Travel Insurance.csv')
df.shape

(63326, 11)

In [5]:
# Display the column name
df.columns 

Index(['Agency', 'Agency Type', 'Distribution Channel', 'Product Name',
       'Claim', 'Duration', 'Destination', 'Net Sales', 'Commision (in value)',
       'Gender', 'Age'],
      dtype='object')

In [6]:
# First five rows of the data
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,NaN,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,NaN,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,NaN,41


In [7]:
# The T is inverting the position of the columns name and the statistics column
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Duration,63326.0,49.317074,101.791566,-2.0,9.0,22.00,53.00,4881.0
Net Sales,63326.0,40.702018,48.845637,-389.0,18.0,26.53,48.00,810.0
Commision (in value),63326.0,9.809992,19.804388,0.0,0.0,0.00,11.55,283.5
Age,63326.0,39.969981,14.017010,0.0,35.0,36.00,43.00,118.0


In [8]:
# The count and data type of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63326 entries, 0 to 63325
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Agency                63326 non-null  object 
 1   Agency Type           63326 non-null  object 
 2   Distribution Channel  63326 non-null  object 
 3   Product Name          63326 non-null  object 
 4   Claim                 63326 non-null  object 
 5   Duration              63326 non-null  int64  
 6   Destination           63326 non-null  object 
 7   Net Sales             63326 non-null  float64
 8   Commision (in value)  63326 non-null  float64
 9   Gender                18219 non-null  object 
 10  Age                   63326 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 5.3+ MB


In [ ]:
df(['Age']).describe()

## Missing Values

In [ ]:
# Checking the columns that consist of null value
# Only the column that return true (for null) will display
df.columns[df.isnull().any()]

In [ ]:
# Also checking the columns that consist of null value
df.isnull().any()

In [ ]:
# Checking the number of null values in each column
df.isnull().sum()

In [ ]:
# Locate the null values in the column Gender
# Maybe too much to fill in idk
df[df["Gender"].isnull()]

In [13]:
# Drop out the missing value
cleandf = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [14]:
# Check the new dataset
cleandf.isnull().sum()

Agency                  0
Agency Type             0
Distribution Channel    0
Product Name            0
Claim                   0
Duration                0
Destination             0
Net Sales               0
Commision (in value)    0
Gender                  0
Age                     0
dtype: int64

In [15]:
cleandf.to_csv("Dataset/Travel Insurance cleaned.csv")

## Outliers

In [9]:
# Function to calculate the outliers

def detect_outliers(df, features):
    outlier_indices = []

    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c],25)
        # 3rd quartile
        Q3 = np.percentile(df[c],75)
        # Interquartile range
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # Detect outlier and their indices
        outlier_list_col = df[(df[c] < Q1 - outlier_step)|(df[c] > Q3 + outlier_step)].index
        # Store indices
        outlier_indices.extend(outlier_list_col)

    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v >2)
    return multiple_outliers

In [10]:
# Perform the function to calculate the outliers
detect_outliers(df,["Age"])

[]

In [16]:
# Locate the outliers
df.loc[detect_outliers(df,["Age"])]

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age


In [12]:
# Dropping the outliers
# We can either drop out the outliers or compute the median and replace the outliers with the median
df_1 = df.drop(detect_outliers(df,["Age"]), axis = 0).reset_index[drop = True]

SyntaxError: invalid syntax (1432681772.py, line 3)